In [69]:
import pandas as pd
from utils.exploratory_data_utils import *

# Importando dados e explorando

## Entendendo os dados e o nosso objetivo:

Nosso objetivo é um modelo de previsão para preços, variáveis categóricas como nomes e ids não vão impactar diretamente na nossa análise, pois não se trata de avaliar o nome do estabelecimento em si, mas sim, o local naqual ele está e se esse local tem influência no preço do estabelecimento por ser um bairro nobre ou não.

In [70]:
data = pd.read_csv('../src/data/bronze/teste_indicium_precificacao.csv')

In [71]:
data.head(3)

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194


In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48894 entries, 0 to 48893
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             48894 non-null  int64  
 1   nome                           48878 non-null  object 
 2   host_id                        48894 non-null  int64  
 3   host_name                      48873 non-null  object 
 4   bairro_group                   48894 non-null  object 
 5   bairro                         48894 non-null  object 
 6   latitude                       48894 non-null  float64
 7   longitude                      48894 non-null  float64
 8   room_type                      48894 non-null  object 
 9   price                          48894 non-null  int64  
 10  minimo_noites                  48894 non-null  int64  
 11  numero_de_reviews              48894 non-null  int64  
 12  ultima_review                  38842 non-null 

* Temos 6 dados categóricos:
  * ultima_review
  * room_type
  * bairro
  * bairro_group
  * host_name
  * nome

In [73]:
data.isna().sum()

id                                   0
nome                                16
host_id                              0
host_name                           21
bairro_group                         0
bairro                               0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimo_noites                        0
numero_de_reviews                    0
ultima_review                    10052
reviews_por_mes                  10052
calculado_host_listings_count        0
disponibilidade_365                  0
dtype: int64

### Retirar colunas nome, id, host_id, host_name e ultima_review

In [74]:
df = data.copy()
df = remove_columns(df, ['id', 'host_id', 'host_name', 'ultima_review', 'nome'])
df.head(3)

,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
1,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,1,365
2,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194


### Analisando coluna com nulos nos reviews_por_mes

In [75]:
# Ou seja, os valores nulos de reviews_por_mes são devido a não haver reviews, logo, vamos preencher com 0
df[(df['reviews_por_mes'].isna()) & (df['numero_de_reviews'] > 0)]
df['reviews_por_mes'] = df['reviews_por_mes'].fillna(0)

In [76]:
df.isna().sum()

bairro_group                     0
bairro                           0
latitude                         0
longitude                        0
room_type                        0
price                            0
minimo_noites                    0
numero_de_reviews                0
reviews_por_mes                  0
calculado_host_listings_count    0
disponibilidade_365              0
dtype: int64

In [77]:
# Não temos dados duplicados
df.duplicated().sum()

np.int64(0)

### Temos mais de 200 bairros, uma possibilidade é agrupar os bairros e colocar uma média de preço para cada bairro

In [78]:
df['bairro'].value_counts()


bairro
Williamsburg          3920
Bedford-Stuyvesant    3714
Harlem                2658
Bushwick              2465
Upper West Side       1971
                      ... 
Richmondtown             1
Fort Wadsworth           1
New Dorp                 1
Rossville                1
Willowbrook              1
Name: count, Length: 221, dtype: int64

In [79]:
nomes_bairros = df['bairro'].value_counts().index
nomes_bairros = pd.DataFrame(nomes_bairros)
nomes_bairros

,bairro
0,Williamsburg
1,Bedford-Stuyvesant
2,Harlem
3,Bushwick
4,Upper West Side
...,...
216,Richmondtown
217,Fort Wadsworth
218,New Dorp
219,Rossville


In [85]:
def get_bairro_mean_price(df: pd.DataFrame, bairro: list) -> pd.DataFrame:
    for b in bairro:
        df['media_bairro'] = df[df["bairro"] == b]["price"].mean()
        return df


In [86]:
df = get_bairro_mean_price(df, nomes_bairros['bairro'].to_list())

In [87]:
df.head(4)

,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,Manhattan,143.802806,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
1,Manhattan,143.802806,40.80902,-73.94190,Private room,150,3,0,0.00,1,365
2,Brooklyn,143.802806,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
3,Manhattan,143.802806,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0
